# Image Classifier Fundamentals

To establish fundamentals, first a set of utilities necessary for image preprocessing and loading from the disk. Then we will use the k-Nearest Neighbor (k-NN) classifier to recognize image class labels using only the raw pixel intensities.

## Dataset: "Animals"

The "Animals" dataset is a simple dataset pieced together by Adrian Rosebrock to reveal the process of training an image classifier using different machine learning techniques. The dataset consists of three class labels, (1) dogs, (2) cats, and (3) pandas. Each class label has exactly 1,000 images, totaling in 3,000 images. The images within the dataset are actually subsets of both the Kaggle "[Dogs vs. Cats](https://www.kaggle.com/c/dogs-vs-cats)" challenge and the [ImageNet](https://www.image-net.org/) dataset. 

## Utilities
**Image Preprocessor**

Images are required to be preprocessed and scaled to have identical widths and heights suitable for machine learning algorithms. Though image preprocessing can take many forms in resizing, scaling, or respect of aspect ratio, this image preprocessor will only resize images and ignore aspect ratio.